In [1]:
import pandas as pd
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
import math
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\vegas_review_reduced.pickle')
df = df.sample(n=20000, random_state=1)

trainset, testset = train_test_split(df, test_size=0.25)

In [3]:
rest_df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\restaurant_in_vegas.pickle')
rest_df = rest_df[['business_id', 'categories']]

In [4]:
cat_trainset = pd.DataFrame(columns=['user_id', 'category', 'stars'])
temp = trainset.merge(rest_df, how='inner', on='business_id')
i=0
for index, row in temp.iterrows():
    categories = row['categories'].split(", ")
    for c in categories:
        cat_trainset.loc[i] = [row['user_id'], c, row['stars']]
        i+=1

In [5]:
cat_data = cat_trainset.copy()

In [7]:
temp2 = testset.merge(rest_df, how='inner', on='business_id')
for index, row in temp2.iterrows():
    categories = row['categories'].split(", ")
    for c in categories:
        cat_data.loc[i] = [row['user_id'], c, row['stars']]
        i+=1

In [26]:
cat_data = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\categories_testing.pickle')

In [19]:
cat_data = cat_data.sample(n=20000, random_state=1)

In [20]:
reader = Reader(rating_scale=(1, 5))
cat_data2 = Dataset.load_from_df(cat_data[['user_id', 'category', 'stars']], reader)
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, cat_data2, measures=['RMSE', 'MAE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')   

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SlopeOne,0.984850,0.596110,0.119629,0.031467
KNNWithZScore,0.988068,0.611777,4.660350,1.463275
KNNWithMeans,0.989905,0.601487,4.322998,1.325323
CoClustering,0.992184,0.634548,1.620576,0.025207
NMF,1.050719,0.776954,1.893413,0.028193
KNNBaseline,1.168486,0.884412,4.167697,1.445557
KNNBasic,1.205626,0.906299,4.063173,1.843688
SVDpp,1.251228,1.018078,1.920397,0.050510
SVD,1.289752,1.058490,1.023424,0.335473


In [21]:
cat_train_data = Dataset.load_from_df(cat_trainset[['user_id', 'category', 'stars']], reader)
cat_train_set = cat_train_data.build_full_trainset()

In [28]:
cat_trainset.to_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\categories_testing_train.pickle')

In [22]:
cat_test_set = testset.merge(rest_df, how='inner', on='business_id')

In [45]:
# [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]
algorithm = CoClustering()
algorithm.fit(cat_train_set)
   
sqr_errors = []
for index, row in cat_test_set.iterrows():
    categories = row['categories'].split(", ")
    predictions = []
    for c in categories:
        predictions.append(algorithm.predict(row['user_id'], c, r_ui=row['stars']).est)
    predicted_rating = sum(predictions)/len(predictions)
    sqr_errors.append((row['stars'] - predicted_rating)**2)

print('test_rmse', math.sqrt(sum(sqr_errors)/len(sqr_errors)))


test_rmse 1.452314676968907


In [ ]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(trainset[['user_id', 'business_id', 'stars']], reader)
train_set = train_data.build_full_trainset()

test_data = Dataset.load_from_df(testset[['user_id', 'business_id', 'stars']], reader)
test_set = test_data.build_full_trainset()
test_set = test_set.build_testset()